## Code for calculating SCC p-values 
Note, this code is for using with Python three. 

In [1]:
# Import the modules we need. 

# !conda install --yes zarr
import xarray as xr
# import zarr as zr
import pandas as pd
import numpy as np

In [2]:
# Load in the draws data, which is a zarr dataset
ds = xr.open_dataset('/shares/gcp/outputs/energy/scc_uncertainty/energy_fulluncertainty_scc_filtered_allpricescenarios_v1.1.nc')
ds

<xarray.Dataset>
Dimensions:     (discrate: 5, pctile: 19, rcp: 2, simulation: 96306, var_type: 8)
Coordinates:
  * simulation  (simulation) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * rcp         (rcp) object 'rcp45' 'rcp85'
  * var_type    (var_type) object 'MERGEETL60' 'REMIND17' ... 'price03'
  * pctile      (pctile) float64 0.05 0.1 0.15 0.2 0.25 ... 0.8 0.85 0.9 0.95
  * discrate    (discrate) float64 0.01 0.02 0.025 0.03 0.05
Data variables:
    scc         (discrate, simulation, rcp, var_type, pctile) float64 ...
Attributes:
    damage_function:            energy_damage_coefficients_TINV_clim_pixel_le...
    damage_function_version:    v1.1, 2020-10-14
    damage_function_author:     Ashwin Rode
    climate_parameters:         istrictrwf_iptdiagnoseemissions_filter_no_his...
    climate_parameter_version:  2019-02-20
    climate_parameter_author:   Kelly McCusker
    climate_output_version:     v2.1, 2019-06-21
    repository:                 https://gitlab.com/ClimateImpactLab/Climate/p...
    tag:                        energy-submission-2020-10-14
    history:                    \nversion 1.1: October 2020 resubmission: pix...
    author:                     Kelly McCusker
    contact:                    kmccusker@rhg.com

In [ ]:
ds.to_dataframe().reset_index()

In [ ]:
def get_p_val(var_type, discrate, rcp, ds):
	
	filtered = ds.sel(var_type = var_type).sel(discrate = discrate).sel(rcp = rcp)

	# Turn it into a dataframe
	df = filtered.to_dataframe().reset_index() 

	# Weight the scc values by their quantile
	df['reps'] = np.where( np.logical_or(df['pctile'] == 0.05,  df['pctile'] == 0.95) , 3, 2)
	df = df.loc[np.repeat(df.index.values, df['reps'] )]

	# Since the scc is below zero, we are going to find the number of draws above zero: 
	df['test_fail'] = np.where(df['scc']>0, 1, 0)

	number_failed = df.test_fail.sum()
	total = df.test_fail.count()

	p = number_failed * 2 / total

	print(p)
	return p

In [ ]:
get_p_val(var_type = 'price014', discrate = 0.03, rcp = 'rcp85', ds = ds)

In [ ]:
get_p_val(var_type = 'price014', discrate = 0.02, rcp = 'rcp85', ds = ds)

In [ ]:
get_p_val(var_type = 'price014', discrate = 0.03, rcp = 'rcp45', ds = ds)

In [ ]:
get_p_val(var_type = 'price014', discrate = 0.02, rcp = 'rcp45', ds = ds)

# I don't trust the above output, so am going to plot the values for rcp 45

In [ ]:
df = ds.sel(var_type = 'price014').sel(discrate = 0.03).sel(rcp = 'rcp45') \
            .to_dataframe().reset_index()
df['reps'] = np.where( np.logical_or(df['pctile'] == 0.05,  df['pctile'] == 0.95) , 3, 2)
df = df.loc[np.repeat(df.index.values, df['reps'] )]
df.head()